In [ ]:
import pandas as pd

In [ ]:
labg = pd.read_parquet('/home/hoyinchan/blue/Data/data2022/UofU/p0_lab_g_UofU.parquet')

In [ ]:
lab = pd.read_parquet('/home/hoyinchan/blue/Data/data2022/UofU/p0_lab_UofU.parquet')

In [ ]:
labg.columns

In [ ]:
lab.columns

In [ ]:
from IPython.display import Audio
sound_file = 'beep-11.wav'
Audio(sound_file, autoplay=True)

In [ ]:
z

In [ ]:
labg = labg[['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'RESULT_NUM', 'RESULT_UNIT', 'DAYS_SINCE_ADMIT']]

In [ ]:
lab = lab[['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'RESULT_NUM', 'RESULT_UNIT', 'DAYS_SINCE_ADMIT']]

In [ ]:
labg = labg.drop_duplicates()

In [ ]:
lab = lab.drop_duplicates()

In [ ]:
labxxx = labg.merge(lab,on=['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'DAYS_SINCE_ADMIT'],how='inner')

In [ ]:
labxyx = labxxx[labxxx['RESULT_UNIT_x']!=labxxx['RESULT_UNIT_y']]

In [ ]:
labxy

In [ ]:
import preprocessing1

In [ ]:
import importlib
importlib.reload(preprocessing1)

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import logging
from sys import getsizeof
import utils_function
import os
import itertools
import pickle5 as pickle

In [ ]:
UCUMunitX = pd.read_csv('UCUMunitX.csv')

In [ ]:
UCUMunitX = UCUMunitX[UCUMunitX['factor_final'].notna()]

In [ ]:
UCUMunitX

In [ ]:
local_custom_convert =  pd.read_csv('local_custom_convert.csv')

In [ ]:
site = 'UofU'

In [ ]:
configs_variables = utils_function.read_config(site)
datafolder = configs_variables['datafolder']
home_directory = configs_variables['home_directory'] 

In [ ]:
loincmap3 =pd.read_csv(home_directory+'loinc/AccessoryFiles/GroupFile/GroupLoincTerms.csv') 

In [ ]:
labtest = lab.copy()

In [ ]:
labtest['site']=site

In [ ]:
labtest2 = labtest.merge(local_custom_convert, left_on = ['LAB_LOINC', 'site', 'RESULT_UNIT'], right_on = ['LAB_LOINC', 'site', 'SOURCE_UNIT'], how='left')
labtest2['NEW_UNIT'] = np.where(labtest2['TARGET_UNIT'].notnull(), labtest2['TARGET_UNIT'], labtest2['RESULT_UNIT'])
labtest2['NEW_RESULT_NUM'] = np.where(labtest2['TARGET_UNIT'].notnull(), labtest2['Multipliyer']*labtest2['RESULT_NUM'], labtest2['RESULT_NUM'])

labtest3 = labtest2.copy()
labtest3['RESULT_UNIT'] = labtest3['NEW_UNIT']
labtest3['RESULT_NUM'] = labtest3['NEW_RESULT_NUM']
labtest3 = labtest3.drop(['NEW_UNIT', 'NEW_RESULT_NUM', 'SOURCE_UNIT', 'TARGET_UNIT', 'LONG_COMMON_NAME', 'Multipliyer'], axis=1)

labtest4 = labtest3.merge(UCUMunitX, on = ['LAB_LOINC', 'RESULT_UNIT'], how='left').copy()
labtest4['NEW_UNIT'] = np.where(labtest4['FINAL_UNIT'].notnull(), labtest4['FINAL_UNIT'], labtest4['RESULT_UNIT'])
labtest4['NEW_RESULT_NUM'] = np.where(labtest4['FINAL_UNIT'].notnull(), labtest4['factor_final']*labtest4['RESULT_NUM'], labtest4['RESULT_NUM'])
labtest4['NEW_LAB_LOINC'] = np.where(labtest4['FINAL_UNIT'].notnull(), labtest4['GroupId'], labtest4['LAB_LOINC'])
labtest4['RESULT_UNIT'] = labtest4['NEW_UNIT']
labtest4['RESULT_NUM'] = labtest4['NEW_RESULT_NUM']
labtest4['LAB_LOINC'] = labtest4['NEW_LAB_LOINC']
labtest4 = labtest4.drop(['GroupId', 'EXAMPLE_UCUM_UNITS',
       'EXAMPLE_UCUM_UNITS_FINAL', 'RESULT_UNIT_CONSENSUS', 'FINAL_UNIT',
       'FINAL_Multiplyer', 'RESULT_UNIT_API', 'FINAL_UNIT_API', 'factor_final',
       'NEW_UNIT', 'NEW_RESULT_NUM', 'NEW_LAB_LOINC'], axis=1)

mmc = loincmap3[loincmap3['Category']=='Mass-Molar conversion'][['GroupId']]
labtest4 = labtest4.merge(mmc, left_on = 'LAB_LOINC', right_on='GroupId', how='left', indicator=True)
labtest4 = labtest4[labtest4['_merge']=='left_only']
labtest4 = labtest4.drop(['GroupId', '_merge'],axis=1)

In [ ]:
UCUMqualX = pd.read_csv('UCUMqualX.csv')
labtest5 = labtest4.copy()
labtest5 = labtest5.merge(UCUMqualX[['LAB_LOINC', 'GroupId']].drop_duplicates(), on='LAB_LOINC', how='left')
labtest5['NEW_LAB_LOINC'] = np.where(labtest5['GroupId'].notnull(), labtest5['GroupId'], labtest5['LAB_LOINC'])
labtest5['LAB_LOINC'] = labtest5['NEW_LAB_LOINC']
labtest5 = labtest5.drop(['GroupId','NEW_LAB_LOINC'],axis=1)
labtest5 = labtest5.drop('site',axis=1)
labtest5 = labtest5.drop_duplicates()

In [ ]:
labtest4 = labtest3.merge(UCUMunitX, on = ['LAB_LOINC', 'RESULT_UNIT'], how='left').copy()

In [ ]:
labtest4['NEW_UNIT'] = np.where(labtest4['FINAL_UNIT'].notnull(), labtest4['FINAL_UNIT'], labtest4['RESULT_UNIT'])
labtest4['NEW_RESULT_NUM'] = np.where(labtest4['FINAL_UNIT'].notnull(), labtest4['factor_final']*labtest4['RESULT_NUM'], labtest4['RESULT_NUM'])
labtest4['NEW_LAB_LOINC'] = np.where(labtest4['FINAL_UNIT'].notnull(), labtest4['GroupId'], labtest4['LAB_LOINC'])

In [ ]:
labtest4['RESULT_UNIT'] = labtest4['NEW_UNIT']
labtest4['RESULT_NUM'] = labtest4['NEW_RESULT_NUM']
labtest4['LAB_LOINC'] = labtest4['NEW_LAB_LOINC']

In [ ]:
labtest4 = labtest4.drop(['GroupId', 'EXAMPLE_UCUM_UNITS',
       'EXAMPLE_UCUM_UNITS_FINAL', 'RESULT_UNIT_CONSENSUS', 'FINAL_UNIT',
       'FINAL_Multiplyer', 'RESULT_UNIT_API', 'FINAL_UNIT_API', 'factor_final',
       'NEW_UNIT', 'NEW_RESULT_NUM', 'NEW_LAB_LOINC'], axis=1)

In [ ]:
labtest4

In [ ]:
mmc = loincmap3[loincmap3['Category']=='Mass-Molar conversion'][['GroupId']]
labtest4 = labtest4.merge(mmc, left_on = 'LAB_LOINC', right_on='GroupId', how='left', indicator=True)
labtest4 = labtest4[labtest4['_merge']=='left_only']
labtest4 = labtest4.drop(['GroupId', '_merge'],axis=1)

In [ ]:
labtest4